In [1]:
import pandas as pd
import numpy as np
import re
import nltk

from googleHelpers import *

youtube = Youtube()

# Getting my top 20 categories by vid count and mins watched from 2018-2021
videoHistory = youtube.videoHistory

videoHistory = videoHistory.drop(
    [
        'Unnamed: 0', 'description', 'viewCount',
    ],
    axis = 1
)

# Adding year for filtering 
videoHistory["searchYear"] = pd.DatetimeIndex(videoHistory.timestamp).year

# Adding a column for duration in units of minutes
duration = []
for index, row in videoHistory.iterrows():
    video_duration = 0
    hours = re.search('\d*H', row['duration'])
    minutes = re.search('\d*M', row['duration'])
    seconds = re.search('\d*S', row['duration'])
    if hours:
        video_duration += (int(hours.group()[ :-1]) * 60)
    if minutes:
        video_duration += int(minutes.group()[ :-1])
    if seconds:
        video_duration += (int(seconds.group()[ :-1]) / 60)
    duration.append(video_duration)
    
videoHistory['duration'] = duration

categoryPopularity = videoHistory.groupby(
    'categoryTitle'
    ).count(
    ).sort_values(
    'id', 
    ascending=False
    ).rename(columns={
    'id':'n'
    })['n'].reset_index()

channelCategories = videoHistory[['categoryTitle', 'channelTitle']].drop_duplicates().merge(
    categoryPopularity,
).groupby('channelTitle').max().reset_index()

channelCategories = channelCategories[
    channelCategories.groupby('channelTitle').n.transform(lambda x: x == x.max())
    ]

videoHistory = videoHistory.drop(
    'categoryTitle', axis=1
).merge(
    channelCategories
)

Loading Credntials From File...


In [2]:
categoryData = videoHistory.groupby(['categoryTitle', 'searchYear']).aggregate(
    np.sum
).rename(
    columns = {
        "duration": "minPlayed"
    }
).drop(
    ['categoryId'], 
    axis=1
)

categoryData = categoryData.assign(
    hrsPlayed=categoryData['minPlayed'] / 60
).drop(
    'minPlayed',
    axis=1
)

timesPlayed = videoHistory.groupby(
    ['categoryTitle', 'searchYear']
).count().rename(columns={
    'id':'videosWatched'
})['videosWatched']

categoryData = categoryData.merge(
    timesPlayed,
    'inner',
    left_on = ['categoryTitle', 'searchYear'],
    right_index = True
)

categoryData.sort_values(
    ['searchYear', 'hrsPlayed', 'videosWatched'],
    ascending=False
).to_csv('tableauData/youtubeCategoryData.csv')

In [3]:
# Getting my top 20 channels by vid count and mins watched from 2018-2021
channelData =  videoHistory.groupby(
    ['channelTitle', 'searchYear']
).aggregate(
    np.sum
).rename(
    columns = {
        "duration": "minPlayed"
    }
).drop(
    ['categoryId', 'n'], 
    axis=1
)

channelData = channelData.assign(
    hrsPlayed=channelData['minPlayed'] / 60
).drop(
    'minPlayed',
    axis=1
).reset_index()

timesPlayed = videoHistory.groupby(
    ['channelTitle', 'searchYear']
).count().rename(columns={
    'id':'videosWatched'
})['videosWatched']

channelData = channelData.merge(
    timesPlayed,
    'inner',
    left_on = ['channelTitle', 'searchYear'],
    right_index = True
)
    
channelData.sort_values(
    ['searchYear', 'hrsPlayed', 'videosWatched'],
    ascending=False
).groupby(
    ['searchYear']
).head(
    50
).merge(
    videoHistory[['channelTitle', 'categoryTitle']].drop_duplicates()
).to_csv('tableauData/youtubeChannelData.csv')

In [4]:
# Getting the Top 20 searches per year from 2018-2021
searchHistory = youtube.searchHistory.copy()
searchHistory["searchYear"] = pd.DatetimeIndex(searchHistory.timestamp).year
searchHistory['timestamp'] = pd.to_datetime(searchHistory['timestamp'])


searchHistory.assign(
    count = 1
).groupby(
    ['query', 'searchYear']
).aggregate(
    np.sum
).drop(
    'Unnamed: 0',
    axis = 1
).sort_values(
    'count',
    ascending=False
).groupby(
    ['searchYear']
).head(
    20
).to_csv('tableauData/youtubeRawSearchCounts.csv')

In [5]:
# Bag of words analysis on youtube searches from December 2020 - Febuary 2021
text = searchHistory.loc[
    (searchHistory['timestamp'] > '2021-01-01') &
    (searchHistory['timestamp'] < '2021-04-01'),
    ['query', 'timestamp']
]

# Tokenize text words
text_corpus = ' '.join(text['query'])
tokens = nltk.word_tokenize(text_corpus)

In [6]:
# Removing stop words and lemmanting words
word_lem=nltk.stem.WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words('english')
lem_tokens = []
for word in tokens:
    if word in stopwords:
        continue
    lem_tokens.append(word_lem.lemmatize(word.lower()))

In [7]:
fdist = nltk.probability.FreqDist()
# Creating a data structure to store word freq
for word in lem_tokens:
    fdist[word.lower()]+=1
# Saving the data into a df and writing to a csv
q1YoutubeSearches = pd.DataFrame().from_dict(dict(fdist), orient = 'index')
q1YoutubeSearches.to_csv('tableauData/youtubeSearches2021q1.csv')

In [8]:
# Pull amazon book purchases from 2021
amzn = {
    "exp20":pd.read_csv('data/amazon2020.csv'),
    "exp21":pd.read_csv('data/amazon2021.csv')
}

# Filtering for book purchases
books20 = amzn['exp20'][amzn['exp20']['Category'] == "ABIS_BOOK"].copy()
books21 = amzn['exp21'][amzn['exp21']['Category'] == "ABIS_BOOK"].copy()

# Books bought in 20 & 21
print("I bought", books20.count().max(), 'books in 2020')
print('I bought', books21.count().max(), 'books in 2021')

# Creating csv files for amazon data
books20.append(books21).to_csv('tableauData/amazonBookPurchases.csv')

I bought 12 books in 2020
I bought 22 books in 2021
